<a href="https://colab.research.google.com/github/klajosw/python/blob/master/kl_py_sqlite_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"> 
    <img src="https://raw.githubusercontent.com/klajosw/python/master/kl_mie_python_logo_250.jpg" 
         align="left" width="251" height="251">
    
</p>


<p> </p>



# Python alapok SQL db használat (Sqlite) 3.

## hibakezelés. xls-be írás

<https://klajosw.blogspot.com/>


---

---

## Kapcsolódás az adatbázishoz

---

In [ ]:
## Modulok impontálása
import sqlite3
import os


---

## SqLite adatbázis lekérdezése és eredmény kiíratása hibakezeléssel

----

In [ ]:
## Minta adatbázis
DB_FILE = r"c:\Users\User\Downloads\Chinook_Sqlite.sqlite"  # adatbázis neve
### forrás : https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite

try:                                ## hibakezelés fiygelés kezdet
    
    con = sqlite3.connect(DB_FILE)  ## kapcsolódás
    cur = con.cursor()              ## kurzor létrehozás
    
## paraméteres lekérdezés 1.
    cur.execute(""" SELECT TrackId, Name, Composer, AlbumId, Bytes  FROM Track WHERE 
    GenreId = ?                           --- sql : paraméter deffinicio
    and Composer is not null  """,
                ('23',))                   ## paraméter érték direct megadása

    print('---- Kiíratás (nyers, csak tartalom) ----\n')
    for row in cur:
        for mezo in row:
            print(mezo, end=' ') # Mezők egy sorba
        print()                  # kiíratás végén sorvégjel kitétele
        
    print()  


    ## lezárások, erőforrás felszabadítás
    cur.close()
    con.close()
    
except sqlite3.Error as e:                   # hiba elkapása
    print ("Hiba történt : ", e.args[0])     # hiba üzenet

---- Kiíratás (nyers, csak tartalom) ----

3375 No Such Thing Chris Cornell 270 3691272 
3376 Poison Eye Chris Cornell 270 3890037 
3377 Arms Around Your Love Chris Cornell 270 3516224 
3378 Safe and Sound Chris Cornell 270 4207769 
3379 She'll Never Be Your Man Chris Cornell 270 3355715 
3380 Ghosts Chris Cornell 270 3799745 
3381 Killing Birds Chris Cornell 270 3588776 
3382 Billie Jean Michael Jackson 270 4606408 
3383 Scar On the Sky Chris Cornell 270 3616618 
3384 Your Soul Today Chris Cornell 270 3385722 
3385 Finally Forever Chris Cornell 270 3565098 
3386 Silence the Voices Chris Cornell 270 4379597 
3387 Disappearing Act Chris Cornell 270 4476203 
3388 You Know My Name Chris Cornell 270 3940651 



In [ ]:
## Minta adatbázis
DB_FILE = r"c:\Users\User\Downloads\Chinook_Sqlite.sqlite"  # adatbázis neve
### forrás : https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite

try:                                ## hibakezelés fiygelés kezdet
    
    con = sqlite3.connect(DB_FILE)  ## kapcsolódás
    cur = con.cursor()              ## kurzor létrehozás
    
## paraméteres lekérdezés 2.
    cur.execute(""" SELECT TrackId, Name, Composer, AlbumId, Bytes  FROM Track WHERE 
    GenreId = :GenreId                    --- sql : nevesített paraméter deffinicio
    and Composer > :Composer """,
    {"GenreId": "23", "Composer": "A"})    ## paraméter szótár érték direct megadása

    print('---- Kiíratás turple (nyers, csak tartalom) ----\n')
    i = cur.fetchone()
    while(i):
        print(i)
        i = cur.fetchone()
        
    print()    
    print('------------------ Második ---------------------')    
    print()    
        
    cur.execute(""" SELECT TrackId, Name, Composer, AlbumId, Bytes  FROM Track WHERE 
    GenreId = :GenreId                    --- sql : nevesített paraméter deffinicio
    and Composer > :Composer """,
    {"GenreId": "24", "Composer": "T"})    ## paraméter szótár érték direct megadása   
    
    print(cur.fetchmany(2))   ## két sor kiírása
    
    print()    
    print('------------------ Harmadik ---------------------')    
    print()  

 
    rows = cur.fetchall()
    for row in rows:
        print("ID     = {} ".format(row[0]))
        print("NEV    = {} ".format(row[1]))
        print("SZERZO = {}".format(row[2]))
        print("BYTE   = {}".format(row[4]))
        print('-------------------------------')
  



    ## lezárások, erőforrás felszabadítás
    cur.close()
    con.close()
    
except sqlite3.Error as e:                   # hiba elkapása
    print ("Hiba történt : ", e.args[0])     # hiba üzenet

---- Kiíratás turple (nyers, csak tartalom) ----

(3375, 'No Such Thing', 'Chris Cornell', 270, 3691272)
(3376, 'Poison Eye', 'Chris Cornell', 270, 3890037)
(3377, 'Arms Around Your Love', 'Chris Cornell', 270, 3516224)
(3378, 'Safe and Sound', 'Chris Cornell', 270, 4207769)
(3379, "She'll Never Be Your Man", 'Chris Cornell', 270, 3355715)
(3380, 'Ghosts', 'Chris Cornell', 270, 3799745)
(3381, 'Killing Birds', 'Chris Cornell', 270, 3588776)
(3382, 'Billie Jean', 'Michael Jackson', 270, 4606408)
(3383, 'Scar On the Sky', 'Chris Cornell', 270, 3616618)
(3384, 'Your Soul Today', 'Chris Cornell', 270, 3385722)
(3385, 'Finally Forever', 'Chris Cornell', 270, 3565098)
(3386, 'Silence the Voices', 'Chris Cornell', 270, 4379597)
(3387, 'Disappearing Act', 'Chris Cornell', 270, 4476203)
(3388, 'You Know My Name', 'Chris Cornell', 270, 3940651)

------------------ Második ---------------------

[(3412, '"Eine Kleine Nachtmusik" Serenade In G, K. 525: I. Allegro', 'Wolfgang Amadeus Mozart', 281, 

---

## SqLite táblák lekérdezése és eredmény excelbe írása

---

In [ ]:
import xlsxwriter
import sqlite3

# Új excel file létrehozása
workbook = xlsxwriter.Workbook(r"c:\Users\User\Downloads\Chinook_file.xlsx")
# Új munkalap létrehozása
worksheet = workbook.add_worksheet()
## Kezdő pozició
row_ = 0
col_ = 0

DB_FILE = r"c:\Users\User\Downloads\Chinook_Sqlite.sqlite"  # adatbázis neve
con = sqlite3.connect(DB_FILE)  ## kapcsolódás
cur = con.cursor()              ## kurzor létrehozás

FIELD_MAX_WIDTH = 30            ## mezők maximális hossz beállítása


cur.execute(""" SELECT t.TrackId, t.Name, t.Composer,  g.name as Style 
FROM Track t INNER JOIN Genre g ON t.GenreId= g.GenreId --- SQL megjegyzés: Join szabály
WHERE t.GenreId=23 and t.Composer is not null           --- SQL megjegyzés: Where feltétel  """)

# Fejléc
for fieldDesc in cur.description:
    print(fieldDesc[0].ljust(FIELD_MAX_WIDTH), end=' ')
    
    ## Excelbe fejléc írása
    worksheet.write(row_, col_, fieldDesc[0])
    col_ += 1
    
print()                                 # Fejléc vége  
for fieldDesc in cur.description:
    print('-' * 30, end=' ')             # Alávonás
print()    # elválasztó sor


# Adattőrzs
fieldIndices = range(len(cur.description))
for row in cur:
    row_ += 1
    col_ = 0
    for fieldIndex in fieldIndices:
        fieldValue = str(row[fieldIndex])
        print(fieldValue.ljust(FIELD_MAX_WIDTH), end=' ')  # Mezők egy sorba
        
        ## excelbe adatok írása
        worksheet.write(row_, col_, row[fieldIndex])
        col_ += 1

    print()                                                 # kiíratás végén sorvégjel kitétele




## lezárások, erőforrás felszabadítás
cur.close()
con.close()
workbook.close()


## Hiányzik: kivétel kezelés, ...

TrackId                        Name                           Composer                       Style                          
------------------------------ ------------------------------ ------------------------------ ------------------------------ 
3375                           No Such Thing                  Chris Cornell                  Alternative                    
3376                           Poison Eye                     Chris Cornell                  Alternative                    
3377                           Arms Around Your Love          Chris Cornell                  Alternative                    
3378                           Safe and Sound                 Chris Cornell                  Alternative                    
3379                           She'll Never Be Your Man       Chris Cornell                  Alternative                    
3380                           Ghosts                         Chris Cornell                  Alternative                    


---
### Adatbázis szerkezete:

<br>
<br>
<p align="left"> 
    <img src="https://1.bp.blogspot.com/-m1j4cr5w_VE/Xu5AoESi7nI/AAAAAAAAe2M/ZuyUsL459bU9qKvtbnG51sfbVUWbsmxVgCK4BGAsYHg/w625-h505/Chinook_Sqlite.jpg" 
         align="left" width="700" height="500">
    
</p>

---